## Project - Movie Revenue Prediction using Linear Regression

In [60]:
import re
from bs4 import BeautifulSoup
import requests
import dateutil.parser
from IPython.core.display import display, HTML
from dataclasses import dataclass
import pathlib
import pandas as pd
import requests
import time

In [61]:
prefix_url = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?'

mpaa_ratings = ['G', 'G/PG', 'PG', 'PG-13', 'R']

url_list_domestic = ['by_mpaa=G', 'by_mpaa=G%2FPG', 'by_mpaa=PG', 'by_mpaa=PG-13', 'by_mpaa=R']

url_list_worldwide = ['area=XWW&by_mpaa=G', 'area=XWW&by_mpaa=G%2FPG',
                      'area=XWW&by_mpaa=PG', 'area=XWW&by_mpaa=PG-13', 'area=XWW&by_mpaa=R']

movies_dataframe = pd.DataFrame()

for page_num in ['', '&offset=200', '&offset=400', '&offset=600', '&offset=800']:

    for index, url in enumerate([prefix_url+x+page_num for x in url_list_domestic]):

        print(url, page_num, index)
        response = requests.get(url)

        page = response.text
        soup = BeautifulSoup(page,"lxml")
        table = soup.find('table')
        if table is not None:
            rows = [row for row in table.find_all('tr')]

            movies = {}
            for row in rows[1:200]:
                items = row.find_all('td')
                link = items[0].find('a')
                title, url = link.text, link['href']
                movies[title] = [url] + [i.text for i in items]

            new_df_movies = pd.DataFrame(movies).T
            movies_dataframe = movies_dataframe.append(new_df_movies)
            print(movies_dataframe.info())

https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G  0
<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, Toy Story 4 to Island of Lemurs: Madagascar
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       199 non-null    object
 1   1       199 non-null    object
 2   2       199 non-null    object
 3   3       199 non-null    object
 4   4       199 non-null    object
 5   5       199 non-null    object
dtypes: object(6)
memory usage: 10.9+ KB
None
https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=G%2FPG  1
<class 'pandas.core.frame.DataFrame'>
Index: 394 entries, Toy Story 4 to Spy Kids
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       394 non-null    object
 1   1       394 non-null    object
 2   2       394 non-null    object
 3   3       394 non-null    object
 4   4       394 non-null    object
 5   5   

In [64]:
movies_dataframe.columns = ['link_stub', 'title', 'rank_g_movies', 'lifetime_gross', 'rank_overall', 'year']

In [65]:
movies_rating_df = movies_dataframe

In [66]:
movies_rating_df.columns = ['link_stub', 'title', 'rank_g_movies', 'lifetime_gross', 'rank_overall', 'year']
movies_rating_df.tail(2) # has all 4357 entries - by rating, domestic 

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year
Oculus,/title/tt2388715/?ref_=bo_cso_table_198,Oculus,998,"$27,695,246","2,931",2014
Saw VI,/title/tt1233227/?ref_=bo_cso_table_199,Saw VI,999,"$27,693,292","2,932",2009


movies_rating_df.head(5)

In [67]:
movies_rating_df.head(5)

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",24,2019
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",28,1994
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",30,2010
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",43,2003
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$290,642,256",100,2001


In [68]:
def money_to_int(moneystring):
    if moneystring is not None:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    else:
        return None

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

In [69]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [70]:
url_test = 'https://www.boxofficemojo.com/title/tt0118715/' 
response = requests.get(url_test)
page = response.text
soup = BeautifulSoup(page, "lxml")

# for link in soup.find_all('a')[:5]:
#     print(link, '\n')
# print(soup.find(class_='mojo-performance-summary-table').prettify())
title_string = soup.find('title').text
dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text
rt_string = soup.find(text=re.compile('Run'))
rt = rt_string.findNext().text
rt = rt.split()
minutes = int(rt[0])*60 + int(rt[2])
print(minutes)# url_test = 'https://www.boxofficemojo.com/title/tt0118715/' 
response = requests.get(url_test)
page = response.text
soup = BeautifulSoup(page, "lxml")

# for link in soup.find_all('a')[:5]:
#     print(link, '\n')
# print(soup.find(class_='mojo-performance-summary-table').prettify())
title_string = soup.find('title').text
dtg = soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text
rt_string = soup.find(text=re.compile('Run'))
rt = rt_string.findNext().text
rt = rt.split()
minutes = int(rt[0])*60 + int(rt[2])
print(minutes)

117
117


In [71]:
runtime = get_movie_value(soup,'Run')
rating = get_movie_value(soup,'MPAA')
budget = get_movie_value(soup,'Budget')
genres = get_movie_value(soup,'Genres')
release_date = get_movie_value(soup,'Release Date')
distributor = get_movie_value(soup,'Domestic Distributor')
release_date = release_date.split('\n')[0]  #Select the only the date
distributor

'Gramercy Pictures (I)See full company information\n\n'

In [72]:
raw_domestic_total_gross = dtg
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Running')
runtime = runtime_to_minutes(raw_runtime)

raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_date = to_date(raw_release_date)

raw_budget = get_movie_value(soup,'Budget')
budget = money_to_int(raw_budget)

genres = get_movie_value(soup,'Genres')
genres

distributor = get_movie_value(soup,'Domestic Distributor')
distributor

'Gramercy Pictures (I)See full company information\n\n'

In [73]:
headers = ['movie title', 'domestic total gross',
           'runtime (mins)', 'rating', 'release date','budget', 'genres','distributor']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date, budget, genres,distributor]))

movie_data.append(movie_dict)
movie_data

[{'movie title': 'Saw VI',
  'domestic total gross': 18034458,
  'runtime (mins)': 117,
  'rating': 'R',
  'release date': datetime.datetime(1998, 3, 6, 0, 0),
  'budget': None,
  'genres': 'Comedy\n    \n        Crime\n    \n        Sport',
  'distributor': 'Gramercy Pictures (I)See full company information\n\n'}]

In [77]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
        - Budget
        - Genres
        - distributor
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    print(url)
    #Request HTML and parse
    response = requests.get(url)
#     print(response.text)
#     time.sleep(1)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    headers = ['movie_title', 'domestic_total_gross',
               'runtime_minutes', 'rating', 'release_date', 'budget','genres','distributor']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()

    #Get domestic gross
    raw_domestic_total_gross = (soup.find(class_='mojo-performance-summary-table')
                                    .find_all('span', class_='money')[0]
                                    .text
                               )
    domestic_total_gross = money_to_int(raw_domestic_total_gross)

    #Get runtime
    raw_runtime = get_movie_value(soup,'Running')
    runtime = runtime_to_minutes(raw_runtime)
    
    #Get rating
    rating = get_movie_value(soup,'MPAA')
    
    #Get budget
    raw_budget = get_movie_value(soup,'Budget')
    budget = money_to_int(raw_budget)
    
    #Get Genres
    genres = get_movie_value(soup,'Genres')
    
    #Get Domestic Distributor
    distributor = get_movie_value(soup,'Domestic Distributor')
    
    #Get release date
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date, budget, genres, distributor]))

    return movie_dict

In [78]:
# movies_1000 = movies_rating_df.head(10)
movies_rating_df_page_info_list = []
for link in movies_rating_df.link_stub:
#     time.sleep(1)
    movies_rating_df_page_info_list.append(get_movie_dict(link))

https://www.boxofficemojo.com/title/tt1979376/?ref_=bo_cso_table_1
https://www.boxofficemojo.com/title/tt0110357/?ref_=bo_cso_table_2
https://www.boxofficemojo.com/title/tt0435761/?ref_=bo_cso_table_3
https://www.boxofficemojo.com/title/tt0266543/?ref_=bo_cso_table_4
https://www.boxofficemojo.com/title/tt0198781/?ref_=bo_cso_table_5
https://www.boxofficemojo.com/title/tt1453405/?ref_=bo_cso_table_6
https://www.boxofficemojo.com/title/tt0120363/?ref_=bo_cso_table_7
https://www.boxofficemojo.com/title/tt0317219/?ref_=bo_cso_table_8
https://www.boxofficemojo.com/title/tt0910970/?ref_=bo_cso_table_9
https://www.boxofficemojo.com/title/tt0114709/?ref_=bo_cso_table_10
https://www.boxofficemojo.com/title/tt0101414/?ref_=bo_cso_table_11
https://www.boxofficemojo.com/title/tt0103639/?ref_=bo_cso_table_12
https://www.boxofficemojo.com/title/tt0382932/?ref_=bo_cso_table_13
https://www.boxofficemojo.com/title/tt0031381/?ref_=bo_cso_table_14
https://www.boxofficemojo.com/title/tt1216475/?ref_=bo_cs

In [79]:
movies_rating_df_page_info_list

[{'movie_title': 'Toy Story 4',
  'domestic_total_gross': 434038008,
  'runtime_minutes': 100,
  'rating': 'G',
  'release_date': datetime.datetime(2019, 6, 20, 0, 0),
  'budget': 200000000,
  'genres': 'Adventure\n    \n        Animation\n    \n        Comedy\n    \n        Family\n    \n        Fantasy',
  'distributor': 'Walt Disney Studios Motion PicturesSee full company information\n\n'},
 {'movie_title': 'The Lion King',
  'domestic_total_gross': 422783777,
  'runtime_minutes': 88,
  'rating': 'G',
  'release_date': datetime.datetime(1994, 6, 15, 0, 0),
  'budget': 45000000,
  'genres': 'Adventure\n    \n        Animation\n    \n        Drama\n    \n        Family\n    \n        Musical',
  'distributor': 'Walt Disney Studios Motion PicturesSee full company information\n\n'},
 {'movie_title': 'Toy Story 3',
  'domestic_total_gross': 415004880,
  'runtime_minutes': 103,
  'rating': None,
  'release_date': datetime.datetime(2010, 6, 16, 0, 0),
  'budget': 200000000,
  'genres': 'Ad

In [80]:
scraped_movies_df = movies_rating_df_page_info_list

In [81]:
scraped_movies_df_dont_delete = movies_rating_df_page_info_list

In [82]:
movies_rating_df_page_info = pd.DataFrame(movies_rating_df_page_info_list)  #convert list of dict to df
movies_rating_df_page_info.set_index('movie_title', inplace=True)

movies_rating_df_page_info

,domestic_total_gross,runtime_minutes,rating,release_date,budget,genres,distributor
movie_title,,,,,,,
Toy Story 4,434038008,100.0,G,2019-06-20,200000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
The Lion King,422783777,88.0,G,1994-06-15,45000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
Toy Story 3,415004880,103.0,None,2010-06-16,200000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
Finding Nemo,380843261,100.0,None,2003-05-30,94000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
"Monsters, Inc.",290642256,92.0,G,2001-11-02,115000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
...,...,...,...,...,...,...,...
Teachers,27774237,107.0,None,1984-10-05,NaN,Comedy\n \n Drama,Metro-Goldwyn-Mayer (MGM)See full company info...
Sinister 2,27740955,97.0,R,2015-08-19,10000000.0,Horror\n \n Mystery\n \n T...,Focus FeaturesSee full company information\n\n
Internal Affairs,27734391,115.0,None,1990-01-12,NaN,Crime\n \n Drama\n \n Thri...,Paramount PicturesSee full company information...


In [83]:
movies_rating_df_page_info

,domestic_total_gross,runtime_minutes,rating,release_date,budget,genres,distributor
movie_title,,,,,,,
Toy Story 4,434038008,100.0,G,2019-06-20,200000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
The Lion King,422783777,88.0,G,1994-06-15,45000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
Toy Story 3,415004880,103.0,None,2010-06-16,200000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
Finding Nemo,380843261,100.0,None,2003-05-30,94000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
"Monsters, Inc.",290642256,92.0,G,2001-11-02,115000000.0,Adventure\n \n Animation\n \n ...,Walt Disney Studios Motion PicturesSee full co...
...,...,...,...,...,...,...,...
Teachers,27774237,107.0,None,1984-10-05,NaN,Comedy\n \n Drama,Metro-Goldwyn-Mayer (MGM)See full company info...
Sinister 2,27740955,97.0,R,2015-08-19,10000000.0,Horror\n \n Mystery\n \n T...,Focus FeaturesSee full company information\n\n
Internal Affairs,27734391,115.0,None,1990-01-12,NaN,Crime\n \n Drama\n \n Thri...,Paramount PicturesSee full company information...


In [84]:
movies_dataframe = movies_dataframe.merge(movies_rating_df_page_info, left_index=True, right_index=True)
print(movies_dataframe.head(50))
movies_dataframe.info()

                                                              link_stub  \
*batteries not included         /title/tt0092494/?ref_=bo_cso_table_122   
*batteries not included         /title/tt0092494/?ref_=bo_cso_table_122   
*batteries not included           /title/tt0092494/?ref_=bo_cso_table_6   
*batteries not included           /title/tt0092494/?ref_=bo_cso_table_6   
10 Cloverfield Lane              /title/tt1179933/?ref_=bo_cso_table_84   
10 Things I Hate About You       /title/tt0147800/?ref_=bo_cso_table_88   
10,000 BC                       /title/tt0443649/?ref_=bo_cso_table_159   
101 Dalmatians                   /title/tt0115433/?ref_=bo_cso_table_27   
101 Dalmatians                   /title/tt0115433/?ref_=bo_cso_table_27   
101 Dalmatians                  /title/tt0115433/?ref_=bo_cso_table_150   
101 Dalmatians                  /title/tt0115433/?ref_=bo_cso_table_150   
102 Dalmatians                   /title/tt0211181/?ref_=bo_cso_table_69   
102 Dalmatians           

In [85]:
movies_dataframe.describe()

,domestic_total_gross,runtime_minutes,budget
count,6.360000e+03,6229.000000,3.437000e+03
mean,7.626366e+07,106.880238,6.265651e+07
std,7.460891e+07,20.374885,5.095598e+07
min,1.737000e+03,61.000000,1.500000e+04
25%,3.236610e+07,93.000000,2.400000e+07
50%,5.160000e+07,103.000000,4.800000e+07
75%,9.314115e+07,116.000000,8.500000e+07
max,8.583730e+08,357.000000,3.560000e+08


In [86]:
movies_dataframe.to_csv(r'Movies_rating_dataset_Sept28.csv', index = False)

In [87]:
movies_rating_df

,link_stub,title,rank_g_movies,lifetime_gross,rank_overall,year
Toy Story 4,/title/tt1979376/?ref_=bo_cso_table_1,Toy Story 4,1,"$434,038,008",24,2019
The Lion King,/title/tt0110357/?ref_=bo_cso_table_2,The Lion King,2,"$422,783,777",28,1994
Toy Story 3,/title/tt0435761/?ref_=bo_cso_table_3,Toy Story 3,3,"$415,004,880",30,2010
Finding Nemo,/title/tt0266543/?ref_=bo_cso_table_4,Finding Nemo,4,"$380,843,261",43,2003
"Monsters, Inc.",/title/tt0198781/?ref_=bo_cso_table_5,"Monsters, Inc.",5,"$290,642,256",100,2001
...,...,...,...,...,...,...
Teachers,/title/tt0088242/?ref_=bo_cso_table_195,Teachers,995,"$27,774,237","2,928",1984
Sinister 2,/title/tt2752772/?ref_=bo_cso_table_196,Sinister 2,996,"$27,740,955","2,929",2015
Internal Affairs,/title/tt0099850/?ref_=bo_cso_table_197,Internal Affairs,997,"$27,734,391","2,930",1990
Oculus,/title/tt2388715/?ref_=bo_cso_table_198,Oculus,998,"$27,695,246","2,931",2014


In [2]:
movies_rating_df = movies_rating_df.merge(movies_rating_df_page_info, left_index=True, right_index=True)
movies_rating_df.head()
movies_rating_df.info()

In [91]:
movies_rating_df.to_csv(r'Movies_rating_data_Sept30.csv', index = False)